In [75]:
host='127.0.0.1'
port=3399
user='root'
passwd='f1'
dbname='ergastdb'

#!/Users/ajh59/anaconda3/bin/pip install pymysql
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://{user}:{passwd}@{host}:{port}/{dbname}'.format(host=host,
                                                                                           port=port,
                                                                                           user=user,
                                                                                           passwd=passwd,
                                                                                           dbname=dbname))


In [330]:
stub='aze'

In [ ]:
name='f1dj_qualiclass_tmp'
ddx.to_sql(name, engine,if_exists='replace')

--------

In [2]:
import lxml.etree
import pandas as pd
import os

def xmlify(fn):
    cmd = 'pdftohtml -xml -nodrm -zoom 1.5 -enc UTF-8 -noframes %s "%s" "%s"' % (
        '',fn+'.pdf', os.path.splitext(fn+'.xml')[0])
    # can't turn off output, so throw away even stderr yeuch
    cmd + " >/dev/null 2>&1"
    os.system(cmd)

def loader(fn):
    fnx=fn+'.xml'
    xmldata = open(fn+'.xml','r').read()
    root = lxml.etree.fromstring(xmldata)
    pages = list(root)
    return pages, root

def loader2018(fn):
    fnx=fn+'.xml'
    #xmldata = open(fn+'.xml','r').read()
    xmldata = open(fn+'.xml','rb').read()
    root = lxml.etree.fromstring(xmldata)
    pages = list(root)
    return pages, root

In [3]:
qclass_fn='/Users/ajh59/Downloads/2018_azerbaijan_grand_prix_2018_final_qualifying_classification_-_doc_26'

In [4]:
loader = loader2018
xmlify(qclass_fn)

In [6]:
pages2,root=loader(qclass_fn)

In [91]:
def flatten(el):           
    result = [ (el.text or "") ]
    for sel in el:
        result.append(flatten(sel))
        result.append(sel.tail or "")
    return "".join(result)

def approx(x,y,t=2):
    if abs(int(x)-int(y))<=t: return True
    return False

import re
def quali_class_table2018(pages):
    rows=[]

    data=[]
    top=0
    left=999
    tmp=''
    tmp_rows={}

    page = pages[1]
    scraping=False
    for el in page:
        txt=flatten(el).strip()
        if not scraping:
            scraping = scraping or txt=='NO DRIVER'
        if not scraping:
            continue


        print(txt,left,scraping)

        if el.attrib['top'] not in tmp_rows:
            tmp_rows[el.attrib['top']]=txt
        else:
            tmp_rows[el.attrib['top']] = "{} {}".format( tmp_rows[el.attrib['top']],txt)

        if scraping and txt.startswith('Timekeeper:'): break #scraping =False
    nn=re.compile(r'^(?P<pos>\d*)\s*(?P<num>\d*)\s*(?P<nameteam>[^:]*)\s(?P<todo>.*)$')
    rows2=[]
    for i in tmp_rows:
        row = tmp_rows[i]
        row2={}
        try:
            row2['pos']=nn.match(row).group('pos')
            row2['num']=nn.match(row).group('num')
            row2['name']=' '.join(nn.match(row).group('nameteam').split()[:2])
            row2['team']=' '.join(nn.match(row).group('nameteam').split()[2:])
            row2['todo']=nn.match(row).group('todo').split()
        except: pass
        if row2: rows2.append(row2)
    rows3=[]
    for a in rows2:
        rows3.append([a['pos'],a['num'],a['name'],a['team']]+a['todo'])
    return pd.DataFrame(rows3) 
        
df=quali_class_table2018(pages2)
df

NO DRIVER 999 True
NAT ENTRANT 999 True
Q1 999 True
LAPS 999 True
% 999 True
TIME 999 True
Q2 999 True
LAPS TIME 999 True
Q3 999 True
LAPS TIME 999 True
1 5 Sebastian VETTEL 999 True
Scuderia Ferrari 999 True
1:42.762 999 True
7 100.218 17:11:50 1:43.015 7 17:30:49 1:41.498 5 17:52:08 999 True
2 44 Lewis HAMILTON 999 True
Mercedes AMG Petronas Motorsport 1:42.693 999 True
7 100.151 17:12:01 1:42.676 7 17:32:45 1:41.677 7 18:00:08 999 True
3 77 Valtteri BOTTAS 999 True
Mercedes AMG Petronas Motorsport 1:43.355 999 True
7 100.796 17:12:09 1:42.679 7 17:28:52 1:41.837 7 18:00:03 999 True
4 3 Daniel RICCIARDO 999 True
Aston Martin Red Bull Racing 999 True
1:42.857 999 True
7 100.311 17:14:41 1:43.482 7 17:29:24 1:41.911 6 18:01:19 999 True
5 33 Max VERSTAPPEN 999 True
Aston Martin Red Bull Racing 999 True
1:42.642 999 True
7 100.101 17:14:37 1:42.901 6 17:29:12 1:41.994 6 18:01:22 999 True
6 7 Kimi RAIKKONEN 999 True
Scuderia Ferrari 999 True
1:42.538 999 True
7 100.000 17:11:55 1:42.510 7

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,,,NO DRIVER,NAT ENTRANT Q1 LAPS % TIME Q2 LAPS TIME Q3 LAPS,TIME,None,None,None,None,None,None,None,None,None
1,1,5,Sebastian VETTEL,Scuderia Ferrari,1:42.762,7,100.218,17:11:50,1:43.015,7,17:30:49,1:41.498,5,17:52:08
2,2,44,Lewis HAMILTON,Mercedes AMG Petronas Motorsport,1:42.693,7,100.151,17:12:01,1:42.676,7,17:32:45,1:41.677,7,18:00:08
3,3,77,Valtteri BOTTAS,Mercedes AMG Petronas Motorsport,1:43.355,7,100.796,17:12:09,1:42.679,7,17:28:52,1:41.837,7,18:00:03
4,4,3,Daniel RICCIARDO,Aston Martin Red Bull Racing,1:42.857,7,100.311,17:14:41,1:43.482,7,17:29:24,1:41.911,6,18:01:19
5,5,33,Max VERSTAPPEN,Aston Martin Red Bull Racing,1:42.642,7,100.101,17:14:37,1:42.901,6,17:29:12,1:41.994,6,18:01:22
6,6,7,Kimi RAIKKONEN,Scuderia Ferrari,1:42.538,7,100.000,17:11:55,1:42.510,7,17:40:37,1:42.490,6,17:52:12
7,7,31,Esteban OCON,Sahara Force India F1 Team,1:43.021,6,100.471,17:09:17,1:42.967,8,17:40:51,1:42.523,6,17:51:42
8,8,11,Sergio PEREZ,Sahara Force India F1 Team,1:43.992,7,101.418,17:15:01,1:43.366,7,17:39:53,1:42.547,6,18:00:19
9,9,27,Nico HULKENBERG,Renault Sport Formula One Team,1:43.746,8,101.178,17:17:16,1:43.232,6,17:40:10,1:43.066,6,18:00:41


In [64]:
tmp_rows

{'263': 'NO DRIVER NAT ENTRANT Q1 LAPS % TIME Q2 LAPS TIME Q3 LAPS TIME',
 '282': '1 5 Sebastian VETTEL Scuderia Ferrari 1:42.762 7 100.218 17:11:50 1:43.015 7 17:30:49 1:41.498 5 17:52:08',
 '300': '2 44 Lewis HAMILTON Mercedes AMG Petronas Motorsport 1:42.693 7 100.151 17:12:01 1:42.676 7 17:32:45 1:41.677 7 18:00:08',
 '318': '3 77 Valtteri BOTTAS Mercedes AMG Petronas Motorsport 1:43.355 7 100.796 17:12:09 1:42.679 7 17:28:52 1:41.837 7 18:00:03',
 '336': '4 3 Daniel RICCIARDO Aston Martin Red Bull Racing 1:42.857 7 100.311 17:14:41 1:43.482 7 17:29:24 1:41.911 6 18:01:19',
 '355': '5 33 Max VERSTAPPEN Aston Martin Red Bull Racing 1:42.642 7 100.101 17:14:37 1:42.901 6 17:29:12 1:41.994 6 18:01:22',
 '373': '6 7 Kimi RAIKKONEN Scuderia Ferrari 1:42.538 7 100.000 17:11:55 1:42.510 7 17:40:37 1:42.490 6 17:52:12',
 '391': '7 31 Esteban OCON Sahara Force India F1 Team 1:43.021 6 100.471 17:09:17 1:42.967 8 17:40:51 1:42.523 6 17:51:42',
 '409': '8 11 Sergio PEREZ Sahara Force India F1

In [88]:
tmp_rows
nn=re.compile(r'^(?P<pos>\d*)\s*(?P<num>\d*)\s*(?P<nameteam>[^:]*)\s(?P<todo>.*)$')
rows2=[]
for i in tmp_rows:
    row = tmp_rows[i]
    row2={}
    try:
        row2['pos']=nn.match(row).group('pos')
        row2['num']=nn.match(row).group('num')
        row2['name']=' '.join(nn.match(row).group('nameteam').split()[:2])
        row2['team']=' '.join(nn.match(row).group('nameteam').split()[2:])
        row2['todo']=nn.match(row).group('todo').split()
    except: pass
    if row2: rows2.append(row2)
rows2

[{'pos': '',
  'num': '',
  'name': 'NO DRIVER',
  'team': 'NAT ENTRANT Q1 LAPS % TIME Q2 LAPS TIME Q3 LAPS',
  'todo': ['TIME']},
 {'pos': '1',
  'num': '5',
  'name': 'Sebastian VETTEL',
  'team': 'Scuderia Ferrari',
  'todo': ['1:42.762',
   '7',
   '100.218',
   '17:11:50',
   '1:43.015',
   '7',
   '17:30:49',
   '1:41.498',
   '5',
   '17:52:08']},
 {'pos': '2',
  'num': '44',
  'name': 'Lewis HAMILTON',
  'team': 'Mercedes AMG Petronas Motorsport',
  'todo': ['1:42.693',
   '7',
   '100.151',
   '17:12:01',
   '1:42.676',
   '7',
   '17:32:45',
   '1:41.677',
   '7',
   '18:00:08']},
 {'pos': '3',
  'num': '77',
  'name': 'Valtteri BOTTAS',
  'team': 'Mercedes AMG Petronas Motorsport',
  'todo': ['1:43.355',
   '7',
   '100.796',
   '17:12:09',
   '1:42.679',
   '7',
   '17:28:52',
   '1:41.837',
   '7',
   '18:00:03']},
 {'pos': '4',
  'num': '3',
  'name': 'Daniel RICCIARDO',
  'team': 'Aston Martin Red Bull Racing',
  'todo': ['1:42.857',
   '7',
   '100.311',
   '17:14:41',


In [89]:
rows3=[]
for a in rows2:
    rows3.append([a['pos'],a['num'],a['name'],a['team']]+a['todo'])
pd.DataFrame(rows3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,,,NO DRIVER,NAT ENTRANT Q1 LAPS % TIME Q2 LAPS TIME Q3 LAPS,TIME,None,None,None,None,None,None,None,None,None
1,1,5,Sebastian VETTEL,Scuderia Ferrari,1:42.762,7,100.218,17:11:50,1:43.015,7,17:30:49,1:41.498,5,17:52:08
2,2,44,Lewis HAMILTON,Mercedes AMG Petronas Motorsport,1:42.693,7,100.151,17:12:01,1:42.676,7,17:32:45,1:41.677,7,18:00:08
3,3,77,Valtteri BOTTAS,Mercedes AMG Petronas Motorsport,1:43.355,7,100.796,17:12:09,1:42.679,7,17:28:52,1:41.837,7,18:00:03
4,4,3,Daniel RICCIARDO,Aston Martin Red Bull Racing,1:42.857,7,100.311,17:14:41,1:43.482,7,17:29:24,1:41.911,6,18:01:19
5,5,33,Max VERSTAPPEN,Aston Martin Red Bull Racing,1:42.642,7,100.101,17:14:37,1:42.901,6,17:29:12,1:41.994,6,18:01:22
6,6,7,Kimi RAIKKONEN,Scuderia Ferrari,1:42.538,7,100.000,17:11:55,1:42.510,7,17:40:37,1:42.490,6,17:52:12
7,7,31,Esteban OCON,Sahara Force India F1 Team,1:43.021,6,100.471,17:09:17,1:42.967,8,17:40:51,1:42.523,6,17:51:42
8,8,11,Sergio PEREZ,Sahara Force India F1 Team,1:43.992,7,101.418,17:15:01,1:43.366,7,17:39:53,1:42.547,6,18:00:19
9,9,27,Nico HULKENBERG,Renault Sport Formula One Team,1:43.746,8,101.178,17:17:16,1:43.232,6,17:40:10,1:43.066,6,18:00:41


In [332]:
session='quali'

In [333]:
qsector_fn='/Users/ajh59/Downloads/2018_04_aze_f1_q0_timing_qualifyingsessionbestsectortimes_v01'

In [334]:
xmlify(qsector_fn)

In [335]:
pages2,root=loader(qsector_fn)

In [341]:
def qualifying_sectors(page):
    scraping=1
    results=[]
    sector=0
    tmp_rows={}
    tmp_rows2={}
    pos=1
    sectors={}
    for el in page:
        if el.tag == "text":
            txt=flatten(el).strip()
            if txt.strip().startswith('Sector'):
                if int(sector): tmp_rows2[sector]=tmp_rows.copy()
                sector=txt.split()[1]
                tmp_rows={}
                pos=0
                
            if scraping:
                if txt.startswith('The F1'): break
                #print(txt)
                
                if el.attrib['top'] not in tmp_rows: 
                    tmp_rows[el.attrib['top']]=txt
                else:
                    tmp_rows[el.attrib['top']] = "{} {}".format( tmp_rows[el.attrib['top']],txt)
                    #print(prev_row)
            else:
                txt=flatten(el).strip()#gettext_with_bi_tags(el)
                txt=txt.strip()
                if txt.startswith("TIME"):
                    scraping=1
                    
    tmp_rows2[sector]=tmp_rows.copy()
    
    tmp_rows3={}
    for k2 in tmp_rows2:
        tmp_rows3[k2]={k:v for k, v in tmp_rows2[k2].items() if not tmp_rows2[k2][k].startswith('Sector')}

    posN ={}
    for k in tmp_rows3['1']:
        posN[k]= tmp_rows3['1'][k].split()[0]
        tmp_rows3['1'][k] = ' '.join(tmp_rows3['1'][k].split()[1:])
    
    nn=re.compile(r'^(?P<num>\d*)\s*(?P<name>\D*)\s*(?P<time>.*)$')
    #sectors[sector]=[]
    
    for sector in tmp_rows3:
        sectors[sector]=[]
        
        for i in tmp_rows3[sector]:
            tmp={}
            row = tmp_rows3[sector][i]
            if i in posN:
                tmp['sector'] = sector
                tmp['pos']=posN[i]
                tmp['num']=nn.match(row).group('num')
                tmp['name']=nn.match(row).group('name')
                tmp['time']=nn.match(row).group('time')
                sectors[sector].append(tmp)

        
    return tmp_rows3,sectors
    #storeSessionQualiSectors(sectorResults)

In [342]:
z,x=qualifying_sectors(pages2[0])
#x

In [343]:
df = pd.concat([pd.DataFrame.from_dict(x['1']),pd.DataFrame.from_dict(x['2']),pd.DataFrame.from_dict(x['3'])])

In [344]:
df['stub']=stub
df['session']=session

In [345]:
df

,name,num,pos,sector,time,stub,session
0,Kimi RAIKKONEN,7,1,1,35.283,aze,quali
1,Lewis HAMILTON,44,2,1,35.296,aze,quali
2,Sebastian VETTEL,5,3,1,35.305,aze,quali
3,Valtteri BOTTAS,77,4,1,35.477,aze,quali
4,Daniel RICCIARDO,3,5,1,35.629,aze,quali
5,Max VERSTAPPEN,33,6,1,35.664,aze,quali
6,Sergio PEREZ,11,7,1,35.708,aze,quali
7,Esteban OCON,31,8,1,35.740,aze,quali
8,Nico HULKENBERG,27,9,1,35.861,aze,quali
9,Fernando ALONSO,14,10,1,36.133,aze,quali


In [346]:
name='f1dj_sectors'
df.to_sql(name, engine,if_exists='replace')

/Users/ajh59/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:165: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)
